#**Analyse et traitement des donnees avec webscraping**

###**Introduction**

##**Importer les packages et les bibliothèques nécessaires :**

In [1]:
## Installation et importation des bibliothèques nécessaires

# Téléchargement du modèle de langue française pour spaCy
!python -m spacy download fr_core_news_sm

# Installation de spaCy
!pip install spacy

# Installation de la bibliothèque NLTK
!pip install nltk

# Installation de la bibliothèque newspaper3k pour le scraping web
!pip install newspaper3k

# Importation des modules nécessaires
import requests  # Pour effectuer des requêtes HTTP
from bs4 import BeautifulSoup  # Pour le scraping HTML
import spacy  # Pour le traitement du langage naturel
import pandas as pd  # Pour la manipulation des données
import matplotlib.pyplot as plt  # Pour la visualisation des données
import seaborn as sns  # Pour des visualisations plus avancées


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 20.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 22.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13538 sha256=

In [2]:
!pip install -U spacy
!python -m spacy download fr_core_news_sm

import spacy

# Charger le modèle de langue française
nlp = spacy.load("fr_core_news_sm")



  Using cached https://github.com/explosion/spacy-models/releases/download/fr_core_news_sm-3.7.0/fr_core_news_sm-3.7.0-py3-none-any.whl (16.3 MB)
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
import spacy
import newspaper
import pandas as pd


# Charger le modèle de traitement du langage naturel français de SpaCy
nlp = spacy.load("fr_core_news_sm")

def extract_triplets(article_text):
    # Initialiser une liste pour stocker les triplets
    triplets = []

    # Analyser le texte de l'article avec SpaCy
    doc = nlp(article_text)

    # Parcourir les phrases du document
    for sentence in doc.sents:
        # Analyser la syntaxe de la phrase
        parsed_sentence = nlp(sentence.text)

        # Extraire les relations sujet-verbe-objet
        for token in parsed_sentence:
            if token.dep_ == "nsubj" and token.head.pos_ == "VERB":
                sujet = token.text
                predicat = token.head.lemma_
                objet = None
                for child in token.head.children:
                    if child.dep_ == "obj":
                        objet = child.text
                        break

                # Ajouter le triplet à la liste
                triplet = f"{sujet} {predicat} {objet}"
                triplets.append(triplet)

    return triplets

def scrape_articles(site_url):
    # Initialiser le scraper de site
    site = newspaper.build(site_url, memoize_articles=False)

    # Récupérer tous les articles du site
    articles = site.articles

    # Liste pour stocker les données des articles
    article_data = []

    # Boucle sur chaque article
    for article in articles:
        try:
            article.download()
            article.parse()

            # Extraire les informations de l'article
            titre = article.title
            contenu = article.text
            date_publication = article.publish_date
            auteurs = article.authors

            # Extraire les triplets du contenu de l'article
            triplets = extract_triplets(contenu)

            # Ajouter les données de l'article à la liste
            article_data.append({
                "titre": titre,
                "contenu": contenu,
                "date": date_publication,
                "auteurs": ", ".join(auteurs),
                "triplets": triplets
            })

        except Exception as e:
            print(f"Erreur lors du traitement de l'article: {e}")
            continue

    # Convertir la liste en DataFrame pandas
    df = pd.DataFrame(article_data)

    # Convertir les dates en un fuseau horaire sans information de fuseau horaire
    df['date'] = df['date'].dt.tz_localize(None)

    # Enregistrer le DataFrame dans un fichier Excel
    df.to_excel("articles_data.xlsx", index=False)

    print("Les données ont été enregistrées dans le fichier 'articles_data.xlsx'.")

# URL du site à parcourir
site_url = "https://www.seneweb.com/"
# Lancer le processus de scraping
scrape_articles(site_url)

CRITICAL:newspaper.network:[REQUEST FAILED] 403 Client Error: Forbidden for url: https://www.seneweb.com/
CRITICAL:newspaper.network:[REQUEST FAILED] 403 Client Error: Forbidden for url: https://www.seneweb.com/rss/


Erreur lors du traitement de l'article: Article `download()` failed with 403 Client Error: Forbidden for url: https://www.seneweb.com/news/Politique/invite-d-ousmane-sonko-jean-luc-melencho_n_440430.html on URL https://www.seneweb.com/news/Politique/invite-d-ousmane-sonko-jean-luc-melencho_n_440430.html
Erreur lors du traitement de l'article: Article `download()` failed with 403 Client Error: Forbidden for url: https://www.seneweb.com/news/Video/l-rsquo-equipe-de-wakana-remporte-la-pre_n_440522.html on URL https://www.seneweb.com/news/Video/l-rsquo-equipe-de-wakana-remporte-la-pre_n_440522.html
Erreur lors du traitement de l'article: Article `download()` failed with 520 Server Error:  for url: https://www.seneweb.com/news/Communique/hausse-supposee-des-factures-d-rsquo-ele_n_440400.html on URL https://www.seneweb.com/news/Communique/hausse-supposee-des-factures-d-rsquo-ele_n_440400.html
Erreur lors du traitement de l'article: Article `download()` failed with 403 Client Error: Forbidden

In [4]:
from google.colab import files
files.download("articles_data.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>